In [13]:
from llama_index.core import Settings, SimpleDirectoryReader ,VectorStoreIndex
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.tools import QueryEngineTool
import json
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.agent.workflow import ReActAgent
from llama_index.core.workflow import Context
from llama_index.core.agent.workflow import AgentStream, ToolCallResult

In [14]:
llm = Ollama(
    model= "gemma3:12b",
    request_timeout=120.0,
    context_window=8128,
    temperature=0.0
)

Settings.llm = llm
Settings.chunk_size=512
Settings.chunk_overlap=64

embed_model = OllamaEmbedding(
    model_name="snowflake-arctic-embed2:latest",
    ollama_additional_kwargs={"mirostat": 0},
)
Settings.embed_model = embed_model

In [15]:
storage_context = StorageContext.from_defaults(persist_dir="./VectorStorage")
index = load_index_from_storage(storage_context=storage_context)
query_engine = index.as_query_engine(llm=Settings.llm,similarity_tok_k=5)
query_engine_tool = QueryEngineTool.from_defaults(query_engine=query_engine, name="RAG_Lookup_tool", description="Query engine tool to look up a knowledge base of documents regarding climate change")


Loading llama_index.core.storage.kvstore.simple_kvstore from ./VectorStorage/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./VectorStorage/index_store.json.


In [ ]:
test_tweet = "Wir als deutche machen sowieso nur 2% des CO2 aussoßes. Warum sollten wir dann was machen???!!!"

In [16]:
class_tweet = "Die sonne scheint einfach mehr, es gib gar keinen klimawandel"

In [17]:
translated_original_tweet =  await llm.acomplete(f"This tweet is in german, translate it into english. Do not include any other words than the tweet: {class_tweet}")

In [18]:
translated_original_tweet

CompletionResponse(text="The sun simply shines more, there's no climate change.", additional_kwargs={'tool_calls': None, 'thinking': None}, raw={'model': 'gemma3:12b', 'created_at': '2025-08-06T06:24:58.836738066Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4658685983, 'load_duration': 4176783591, 'prompt_eval_count': 44, 'prompt_eval_duration': 264047719, 'eval_count': 14, 'eval_duration': 217286037, 'message': Message(role='assistant', content="The sun simply shines more, there's no climate change.", thinking=None, images=None, tool_calls=None), 'usage': {'prompt_tokens': 44, 'completion_tokens': 14, 'total_tokens': 58}}, logprobs=None, delta=None)

In [19]:
prompt = f"""
        This is a tweet from Twitter:
        "{translated_original_tweet.text}"
        Workflow: 
        2) Extract the claims in the tweet. If you don't find any relevant data answer on common knowledge.
        3) Use the RAG_Lookup_tool to fact check the claims in the tweet
        4) If the tweet contains wrong information, write an answer to the tweet in english, where you correct the wrong claims Be direct and critizise missinformation. If the tweet has no claims or all claims are correct answe with: NO_ACTION_NEEDED
        """

In [20]:
from llama_index.core.workflow.errors import WorkflowRuntimeError

In [21]:
try:    
    agent = ReActAgent(tools=[query_engine_tool])
    ctx = Context(agent)
    handler = agent.run(prompt, ctx=ctx, max_iterations=30)

    async for ev in handler.stream_events():
        # if isinstance(ev, ToolCallResult):
        #     print(f"\nCall {ev.tool_name} with {ev.tool_kwargs}\nReturned: {ev.tool_output}")
        if isinstance(ev, AgentStream):
            print(f"{ev.delta}", end="", flush=True)

    response = await handler
    ro= response.model_dump()
    answer_tweet = ro["response"]["blocks"][0]["text"]
except WorkflowRuntimeError as e:
    print(f"Error catched {e}")

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: RAG_Lookup_tool
Action Input: {"input": "The sun simply shines more, there's no climate change."}
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: That's incorrect. Climate change is a real and serious threat, supported by overwhelming scientific evidence. The idea that it's simply the sun shining more is a dangerous myth that ignores decades of research.

In [22]:
answer_tweet

"That's incorrect. Climate change is a real and serious threat, supported by overwhelming scientific evidence. The idea that it's simply the sun shining more is a dangerous myth that ignores decades of research."

In [23]:
final_response = await llm.acomplete(f"If this tweet is in english, translate it into german if not, repeat the tweet. Do not include any other words than the tweet: {answer_tweet}")

In [24]:
final_response.text

'Das ist falsch. Der Klimawandel ist eine reale und ernste Bedrohung, die durch überwältigende wissenschaftliche Beweise belegt wird. Die Vorstellung, dass es einfach nur die Sonne ist, die stärker scheint, ist ein gefährlicher Mythos, der jahrzehntelange Forschung ignoriert.'